In [1]:
# | default_exp preprocess.dataloader

In [2]:
import logging
from typing import Optional

# | export
import numpy as np
import pytorch_lightning as pl
import torch
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import DataLoader, Dataset

logger = logging.getLogger(__name__)

In [3]:
# | export
import logging
import warnings

# Configure logging
logging.basicConfig(level=logging.WARNING)  # Change to DEBUG for more details

logger = logging.getLogger(__name__)

In [4]:
# | hide


from fastcore.test import test_eq
from nbdev.showdoc import show_doc

In [5]:
# | export
device = "cuda" if torch.cuda.is_available() else "cpu"
torch._dynamo.config.suppress_errors = True

In [6]:
# | export


class TSRegressionDataset(Dataset):
    def __init__(self, df, in_features, out_features, window=1):
        super(TSRegressionDataset, self).__init__()

        self.data = df
        self.in_features = in_features
        self.out_features = out_features
        self.window = window

        self.groups = {uid: group.reset_index(drop=True) for uid, group in df.groupby("unique_id")}

    def __len__(self):
        assert self.in_features + self.out_features < len(
            self.data
        ), f"in_features + out_features should not be greater than series len {self.data.shape[0]}"

        return sum(
            [
                (len(g) - (self.in_features + self.out_features)) // self.window + 1
                for g in self.groups.values()
            ]
        )

    def __getitem__(self, idx):

        for uid, df in self.groups.items():
            max_idx = (len(df) - (self.in_features + self.out_features)) // self.window + 1

            if idx < max_idx:
                start = idx * self.window
                x = torch.tensor(
                    df["y"].iloc[start : start + self.in_features].values,
                    dtype=torch.float32,  # device=torch.device("cuda")
                )
                y = torch.tensor(
                    df["y"]
                    .iloc[start + self.in_features : start + self.in_features + self.out_features]
                    .values,
                    dtype=torch.float32,  # device=torch.device("cuda")
                )
                return x, y

            idx -= max_idx

In [7]:
# | export


class TSDataLoader(pl.LightningDataModule):
    def __init__(self, df, in_features, out_features, window=1, batch_size=32):
        super(TSDataLoader, self).__init__()
        self.data = df
        self.in_features = in_features
        self.out_features = out_features
        self.window = window
        self.batch_size = batch_size

    def setup(self, stage=None):
        assert list(self.data.columns) == [
            "unique_id",
            "ds",
            "y",
        ], "Columns must be ['unique_id', 'ds', 'y']"

        self.data = self.data.sort_values(["unique_id", "ds"])

        # Train / Val / Test Split (70/15/15)
        train_sz = int(len(self.data) * 0.7)
        val_sz = int(len(self.data) * 0.15)

        self.train_df = self.data.iloc[:train_sz]
        self.val_df = self.data.iloc[train_sz : train_sz + val_sz]
        self.test_df = self.data.iloc[train_sz + val_sz :]

        self.train = TSRegressionDataset(
            self.train_df, self.in_features, self.out_features, self.window
        )
        self.val = TSRegressionDataset(
            self.val_df, self.in_features, self.out_features, self.window
        )
        self.test = TSRegressionDataset(
            self.test_df, self.in_features, self.out_features, self.window
        )

    def train_dataloader(self):
        return DataLoader(
            self.train,
            batch_size=self.batch_size,
            shuffle=True,
            pin_memory=True,
            num_workers=32,
            persistent_workers=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val,
            batch_size=self.batch_size,
            shuffle=False,
            pin_memory=True,
            num_workers=32,
            persistent_workers=True,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test,
            batch_size=self.batch_size,
            shuffle=False,
            pin_memory=True,
            num_workers=32,
            persistent_workers=True,
        )

In [8]:
# | export
if __name__ == "__main__":
    from neuralforecast.utils import AirPassengersDF as df

    in_features, out_features, window, batch_sz = 12, 6, 1, 32
    ds = TSDataLoader(df, in_features, out_features, window, batch_sz)
    ds.setup()
    for features, labels in ds.train_dataloader():
        print(features.shape, labels.shape)

torch.Size([32, 12]) torch.Size([32, 6])
torch.Size([32, 12]) torch.Size([32, 6])
torch.Size([19, 12]) torch.Size([19, 6])


In [9]:
# | export


class UnivariateTSDataset(Dataset):
    def __init__(self, windows, device: Optional[str] = None):
        """
        Args:
            windows: List of (x, y) window tuples
            device: If specified, pre-load data to this device (e.g., 'cuda')
        """
        # Store as float32 numpy arrays for memory efficiency
        self.x = np.stack([w[0] for w in windows], axis=0).astype(np.float32)
        self.y = np.stack([w[1] for w in windows], axis=0).astype(np.float32)

        # Optionally pre-load to GPU
        self.device = device
        if device:
            self.x_tensor = torch.from_numpy(self.x).to(device)
            self.y_tensor = torch.from_numpy(self.y).to(device)
        else:
            self.x_tensor = self.y_tensor = None

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        if self.device:
            # Return pre-loaded GPU tensors if available
            return self.x_tensor[idx], self.y_tensor[idx]
        else:
            # Otherwise convert to tensor on the fly with pin_memory
            return (torch.from_numpy(self.x[idx]), torch.from_numpy(self.y[idx]))


class UnivariateTSDataModule(pl.LightningDataModule):
    def __init__(
        self,
        df,
        input_size,
        horizon,
        batch_size=32,
        num_workers=8,  # Optimal for most systems
        train_split=0.7,
        val_split=0.15,
        normalize=True,
        scaler_type="minmax",
        split_type="horizontal",
        step_size=1,
        pin_memory=True,  # Enable for GPU training
        prefetch_factor=2,  # Prefetch batches to GPU
        persistent_workers=True,  # Maintain workers between epochs
        gpu_preload=False,  # Preload data to GPU if memory allows
    ):
        super().__init__()
        self.save_hyperparameters(ignore=["df"])

        self.df = df
        self.input_size = input_size
        self.horizon = horizon
        self.batch_size = batch_size
        self.num_workers = min(num_workers, torch.get_num_threads())  # Don't exceed system threads
        self.train_split = train_split
        self.val_split = val_split
        self.normalize = normalize
        self.scaler_type = scaler_type
        self.split_type = split_type
        self.step_size = step_size
        self.scalers = {}
        self.pin_memory = pin_memory
        self.prefetch_factor = prefetch_factor
        self.persistent_workers = persistent_workers
        self.gpu_preload = gpu_preload
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Validate splits
        if not 0 < train_split + val_split <= 1:
            raise ValueError("train_split + val_split must be between 0 and 1")
        if step_size < 1:
            raise ValueError("step_size must be >= 1")

    def _generate_windows(self, series):
        """Vectorized window generation for better CPU utilization"""
        series_len = len(series)
        if series_len < self.input_size + self.horizon:
            return []

        max_idx = series_len - self.input_size - self.horizon + 1
        if max_idx <= 0:
            return []

        # Create index ranges for vectorized operations
        window_starts = np.arange(0, max_idx, self.step_size, dtype=np.int32)
        window_ends = window_starts + self.input_size
        horizon_ends = window_ends + self.horizon

        # Ensure we don't go out of bounds
        valid_windows = horizon_ends <= series_len
        window_starts = window_starts[valid_windows]
        window_ends = window_ends[valid_windows]
        horizon_ends = horizon_ends[valid_windows]

        # Vectorized window creation
        x_windows = np.lib.stride_tricks.sliding_window_view(series, window_shape=self.input_size)[
            window_starts
        ]

        y_windows = np.stack(
            [
                series[window_end:horizon_end]
                for window_end, horizon_end in zip(window_ends, horizon_ends)
            ]
        )

        return list(zip(x_windows, y_windows))

    def setup(self, stage=None):
        # Process data in chunks to reduce memory usage
        grouped = self.df.groupby("unique_id")

        # Initialize lists to store windows
        train_windows, val_windows, test_windows = [], [], []

        # Process each series individually to minimize peak memory
        for unique_id, group in grouped:
            series = group["y"].values.astype(np.float32)

            if self.normalize:
                if self.scaler_type == "minmax":
                    scaler = MinMaxScaler()
                elif self.scaler_type == "standard":
                    scaler = StandardScaler()
                else:
                    raise ValueError("scaler_type must be 'minmax' or 'standard'")

                series = scaler.fit_transform(series.reshape(-1, 1)).flatten()
                self.scalers[unique_id] = scaler

            windows = self._generate_windows(series)
            if not windows:
                logger.warning(f"{unique_id} - Series too short for windowing")
                continue

            if self.split_type == "horizontal":
                num_windows = len(windows)
                train_end = int(num_windows * self.train_split)
                val_end = train_end + int(num_windows * self.val_split)

                train_windows.extend(windows[:train_end])
                val_windows.extend(windows[train_end:val_end])
                test_windows.extend(windows[val_end:])

            elif self.split_type == "vertical":
                # Vertical split handled in second pass below
                pass
            else:
                raise ValueError("split_type must be 'horizontal' or 'vertical'")

        # Handle vertical split if needed
        if self.split_type == "vertical":
            unique_ids = list(grouped.groups.keys())
            np.random.shuffle(unique_ids)
            total_series = len(unique_ids)
            train_end = int(total_series * self.train_split)
            val_end = train_end + int(total_series * self.val_split)

            train_ids = unique_ids[:train_end]
            val_ids = unique_ids[train_end:val_end]
            test_ids = unique_ids[val_end:]

            # Second pass for vertical split
            for unique_id, group in grouped:
                if unique_id not in self.scalers:  # Skip if already processed
                    series = group["y"].values.astype(np.float32)
                    windows = self._generate_windows(series)
                    if not windows:
                        continue

                if unique_id in train_ids:
                    train_windows.extend(windows)
                elif unique_id in val_ids:
                    val_windows.extend(windows)
                elif unique_id in test_ids:
                    test_windows.extend(windows)

        # Create datasets with optional GPU pre-loading
        self.train_dataset = UnivariateTSDataset(
            train_windows, device=self.device if self.gpu_preload else None
        )
        self.val_dataset = UnivariateTSDataset(
            val_windows, device=self.device if self.gpu_preload else None
        )
        self.test_dataset = UnivariateTSDataset(
            test_windows, device=self.device if self.gpu_preload else None
        )

        logger.info(
            f"Train windows: {len(self.train_dataset)}, "
            f"Val windows: {len(self.val_dataset)}, "
            f"Test windows: {len(self.test_dataset)}"
        )

    def _create_dataloader(self, dataset, shuffle=False):
        return DataLoader(
            dataset,
            batch_size=self.batch_size,
            shuffle=shuffle,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory and not self.gpu_preload,  # No need if preloaded
            prefetch_factor=self.prefetch_factor,
            persistent_workers=self.persistent_workers,
            drop_last=shuffle,  # Drop last batch only for training
        )

    def train_dataloader(self):
        return self._create_dataloader(self.train_dataset, shuffle=True)

    def val_dataloader(self):
        return self._create_dataloader(self.val_dataset)

    def test_dataloader(self):
        return self._create_dataloader(self.test_dataset)

    def inverse_transform(self, data, unique_id):
        if self.normalize and unique_id in self.scalers:
            return self.scalers[unique_id].inverse_transform(data.reshape(-1, 1)).flatten()
        return data